In [1]:
# kakao open api에서 키워드 검색을 통해 주소와 위도 경도를 찾을 수 있으나
# 키워드와 관련된 장소가 모두 검색되어 위도 경도를 분류하는 작업을 1번 더 해야 하므로
# 도로명 주소를 입력받아 위도 경도를 구하도록 한다

In [2]:
def search_road_addr():
    # 네이버 openAPI 검색 > 지역 활용
    # 키워드를 통해 도로명 주소를 얻음
    
    import urllib.request
    import json
    from urllib import parse
    
    #애플리케이션 클라이언트 id 및 secret
    client_id = "your_naver_application_client_id" # 네이버 developers에서 발급받은 client_id
    client_secret = "your_naver_application_client_secret" # 네이버 developers에서 발급받은 client_secret
    
    #지역검색 url
    url = "https://openapi.naver.com/v1/search/local.json"
    option = "&display=1&start=1&sort=random" # sort: random(유사도 순), comment(카페/블로그 리뷰 개수 순)
    keyword = input("키워드: ")
    q_keyword = urllib.parse.quote(keyword)
    query = "?query="+q_keyword
    url_query = url + query + option
    
    #Open API 검색 요청 개체 설정
    request = urllib.request.Request(url_query)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    
    #검색 요청 및 처리
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode == 200):
        response_body = response.read()
        content = response_body.decode('utf-8')
        words = content.replace("\n\t\t\t", '').split(',') # 내용을 ,로 분류
        words_index = [i for i in range(len(words)) if 'roadAddress' in words[i]][0] # 도로명 주소에 해당하는 인덱스 검색
        word = words[words_index].replace('"','') # 내용에서 도로명 주소에 해당하는 키와 값 가져오기
        roadaddr = word.strip('roadAddress:')
        search_addr = [keyword, roadaddr]
        return search_addr
    else:
        return None

In [3]:
def road_addr(): # 위도 경도 구하기
    # kakao api에서 키워드 검색을 통해 주소와 위도 경도를 찾을 수 있으나
    # 키워드와 관련된 장소가 모두 검색되어 위도 경도를 분류하는 작업을 1번 더 해야 하므로
    # 도로명 주소를 입력받아 위도 경도를 구하도록 한다
    
    import json
    import requests
    import math
    from bs4 import BeautifulSoup as bs
    
    # 도로명 주소 함수 호출
    s_addr =  search_road_addr()
    
    # 키워드 추출
    keyword = s_addr[0]
    
    # 도로명 주소 구하기
    addr = s_addr[1]
    
    # 카카오 map api 주소?query=도로명 주소
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query='+addr
    
    # api 신청 시 발급받은 RestAPI 키
    API_key = 'RestAPI_key'
    headers = {"Authorization": "KakaoAK " + API_key}
    
    # api를 통해 가져온 결과
    result = json.loads(str(requests.get(url,headers=headers).text))
    
    # 결과에서 도로명 주소가 담긴 인덱스 값 추출
    road_addr = result['documents'][0]['road_address']
    
    # road_addr['x']는 소수점을 포함한 문자열이므로 숫자로 변환하기 위해서는 int가 아닌 float로 변환해야 한다
    # latitude: 위도
    # longitude: 경도
    latitude = math.trunc(float(road_addr['x']))
    longitude = math.trunc(float(road_addr['y']))
    
    # 검색 결과를 리스트에 담아줌
    keyword_result = [keyword, addr, latitude, longitude]
    
    return keyword_result

In [4]:
results = road_addr()
print(results[0] + 
      '으로 입력하신 결과 도로명 주소: ' + results[1] + 
      ', 위도: ' + str(results[2]) + 
      ', 경도: ' + str(results[3])
)

키워드:하이미디어 종로
하이미디어 종로으로 입력하신 결과 도로명 주소: 서울특별시 종로구 인사동길 12 대일빌딩 7층, 위도: 126, 경도: 37
